In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import os
import sys
import matplotlib.image as img
import scipy.io
import pickle
from sklearn.metrics import pairwise_distances, classification_report, confusion_matrix, roc_auc_score
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

If the following code doesn't run, then do 'pip install ipynb' in the command line. This code lets us import functions from notebooks in the lib folder. Lib is supposed to have all the model training/predicting functions and the doc folder is only supposed to have report/presentation files like main.ipynb.

In [2]:
import ipynb
sys.path.append('../lib/')

### Step 0: set work directories

In [3]:
np.random.seed(2020)

Provide directories for training images. Training images and Training fiducial points will be in different subfolders.

In [4]:
#Change train_dir to where you have the train set because we can't
#upload the train set to github

train_dir = "/Users/rohan/Desktop/train_set/"
train_image_dir = train_dir+"images/"
train_pt_dir = train_dir+"points/"
train_label_path = train_dir+"label.csv"

### Step 1: set up controls for evaluation experiments

In this chunk, we have a set of controls for the evaluation experiments. 

+ (T/F) cross-validation on the training set
+ (T/F) reweighting the samples for training set 
+ (number) K, the number of CV folds
+ (T/F) process features for training set
+ (T/F) run evaluation on an independent test set
+ (T/F) process features for test set

In [5]:
run_cv = True # run cross-validation on the training set
sample_reweight = True # run sample reweighting in model training
K = 5  # number of CV folds
run_feature_train = True # process features for training set
run_test = True # run evaluation on an independent test set
run_feature_test = True # process features for test set

Set these to be False if you don't want to go through training certain models when running the notebook.

In [6]:
run_baseline = True
run_baseline_pca = True
run_knn = True

### Step 2: import data and train-test split

In [7]:
info = pd.read_csv(train_label_path)
n = info.shape[0]

#Data is imbalanced 
print('Number of records with label 0 (basic emotion): {:4d} '.format(info.loc[info['label']==0].shape[0]))
print('Number of records with label 1 (complex emotion): {:2d} '.format(info.loc[info['label']==1].shape[0]))

Number of records with label 0 (basic emotion): 2402 
Number of records with label 1 (complex emotion): 598 


In [8]:
#we could use sklearn train_test_split here instead of doing it
#manually like in the starter code
n_train = int(round(n*(4/5),0))
train_idx = np.random.choice(list(info.index),size=n_train,replace=False)
test_idx = list(set(list(info.index))-set(train_idx)) #set difference

Reading images (we never use the image list later since we're just using the fiducial points for the features)

In [9]:
n_files = len(os.listdir(train_image_dir))

image_list = []
for i in range(1,101): # 1 to 100
    image = img.imread(train_image_dir+'{:04d}'.format(i)+'.jpg')
    image_list.append(image)

Fiducial points are stored in matlab format. In this step, we read them and store them in a list.

In [10]:
#function to read fiducial points
#input: index
#output: matrix of fiducial points corresponding to the index

def readMat_matrix(index):
    try:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinatesUnwarped']
    except KeyError:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinates2']
    return np.matrix.round(mat_data,0)

#load fiducial points
#pickle is the closest equivalent to .RData that I could find in Python
fiducial_pt_list = list(map(readMat_matrix,list(range(1,n_files+1))))
pickle.dump(fiducial_pt_list, open( "../output/fiducial_pt_list.p", "wb" ) )

### Step 3: construct features and responses

`feature.R` should be the wrapper for all your feature engineering functions and options. The function `feature( )` should have options that correspond to different scenarios for your project and produces an R object that contains features and responses that are required by all the models you are going to evaluate later. 
  
  + `feature.R`
  + Input: list of images or fiducial point
  + Output: an RData file that contains extracted features and corresponding responses

In [11]:
# Use feature.ipynb's feature function to generate features for the train and test points

from ipynb.fs.full.feature import feature

tm_feature_train = np.nan
if run_feature_train == True:
    start = time.time()
    dat_train = feature(fiducial_pt_list, train_idx, info)
    end = time.time()
    tm_feature_train = end-start
    pickle.dump(dat_train, open( "../output/train_data.p", "wb" ) )
else:
    dat_train = pickle.load(open("../output/train_data.p", "rb"))
    
    
tm_feature_test = np.nan
if run_feature_test == True:
    start = time.time()
    dat_test = feature(fiducial_pt_list, test_idx, info)
    end = time.time()
    tm_feature_test = end-start
    pickle.dump(dat_test, open( "../output/test_data.p", "wb" ) )
else:
    dat_test = pickle.load(open("../output/test_data.p", "rb"))
    

In [12]:
#Get the traning/test features and labels

feature_train = dat_train.loc[:, dat_train.columns != 'labels']
label_train = dat_train['labels']

feature_test = dat_test.loc[:, dat_test.columns != 'labels']
label_test = dat_test['labels']

In [13]:
#PCA (not sure if i did it correctly)

scaler = MinMaxScaler()
feature_train_scaled = scaler.fit_transform(feature_train)

#pick the number of components that captures 95% of the variance
pca = PCA(n_components = 0.95).fit(feature_train_scaled)
feature_train_PCA = pca.transform(feature_train_scaled)
feature_test_PCA = pca.transform(feature_test)

### Step 4: Train a classification model with training features and responses

Call the train model and test model from library. 

`train.R` and `test.R` should be wrappers for all your model training steps and your classification/prediction steps. 

+ `train.R`
  + Input: a data frame containing features and labels and a parameter list.
  + Output:a trained model
+ `test.R`
  + Input: the fitted classification model using training data and processed features from testing images 
  + Input: an R object that contains a trained classifier.
  + Output: training model specification

+ In this Starter Code, we use logistic regression with LASSO penalty to do classification.

#### Model selection with cross-validation
* Do model selection by choosing among different values of training model parameters.

### Baseline Model

In [14]:
if run_baseline == True:
    
    lmbd = [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
    model_labels = ["Gradient Boosting with learning rate = "+str(x) for x in lmbd]
    
    #need to do a grid search for optimal parameters
    #takes a really long time and we don't know the features yet
    #so i commented the grid search out for now
    params = {'learning_rate':lmbd, 'max_depth': [1,2,3,4], 'n_estimators':[100,200,300,400,500]}
    #gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=K).fit(feature_train,label_train)

    #Baseline model
    #need to do grid search to get optimal parameters though
    start = time.time()
    gbm = GradientBoostingClassifier(learning_rate=0.1,max_depth=2,n_estimators=100)
    gbm.fit(feature_train,label_train)
    end = time.time()
    print('Training time: {:4f} seconds'.format(end-start))
    
    start = time.time()
    test_preds = gbm.predict(feature_test)
    end = time.time()
    print('Prediction time: {:4f} seconds'.format(end-start))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}'.format(1-classification_error)) #same as gbm.score(feature_test,label_test)
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_preds)))

Training time: 81.052073 seconds
Prediction time: 0.049767 seconds

Classification Error: 0.188333
Accuracy: 0.811667
Classification Report:

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       473
           1       0.75      0.17      0.27       127

    accuracy                           0.81       600
   macro avg       0.78      0.58      0.58       600
weighted avg       0.80      0.81      0.76       600

Confusion Matrix:

[[466   7]
 [106  21]]

AUC: 0.575278


### Baseline Model With PCA

In [15]:
#something wrong with PCA since we're getting AUC of 0.5
if run_baseline_pca == True:
    
    lmbd = [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
    model_labels = ["Gradient Boosting with learning rate = "+str(x) for x in lmbd]
    
    #need to do a grid search for optimal parameters
    #takes a really long time and we don't know the features yet
    #so i commented the grid search out for now
    params = {'learning_rate':lmbd, 'max_depth': [1,2,3,4], 'n_estimators':[100,200,300,400,500]}
    #gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=K).fit(feature_train,label_train)

    #Baseline model with PCA
    #need to do grid search to get optimal parameters though
    start = time.time()
    gbm_pca=GradientBoostingClassifier(learning_rate=0.1,max_depth=2,n_estimators=100)
    gbm_pca.fit(feature_train_PCA,label_train)
    end = time.time()
    print('Training time: {:4f} seconds'.format(end-start))
    
    start = time.time()
    test_preds = gbm_pca.predict(feature_test_PCA)
    end = time.time()
    print('Prediction time: {:4f} seconds'.format(end-start))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_preds)))

Training time: 2.045840 seconds
Prediction time: 0.000811 seconds

Classification Error: 0.788333
Accuracy: 0.211667

Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       473
           1       0.21      1.00      0.35       127

    accuracy                           0.21       600
   macro avg       0.11      0.50      0.17       600
weighted avg       0.04      0.21      0.07       600

Confusion Matrix:

[[  0 473]
 [  0 127]]

AUC: 0.500000


/Users/rohan/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### KNN Model

In [16]:
if run_knn == True:
    
    start = time.time()
    #need to cross validate to pick best value (after finalizing features)
    knn = KNeighborsClassifier(n_neighbors = 8)
    knn.fit(feature_train,label_train)
    end = time.time()
    print('Training time: {:4f} seconds'.format(end-start))
    
    start = time.time()
    test_preds = knn.predict(feature_test)
    end = time.time()
    print('Prediction time: {:4f} seconds'.format(end-start))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test)) #Classification Error
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_preds)))

Training time: 0.804046 seconds
Prediction time: 12.661891 seconds

Classification Error: 0.208333
Accuracy: 0.791667

Classification Report:

              precision    recall  f1-score   support

           0       0.80      0.98      0.88       473
           1       0.55      0.09      0.16       127

    accuracy                           0.79       600
   macro avg       0.67      0.54      0.52       600
weighted avg       0.75      0.79      0.73       600

Confusion Matrix:

[[463  10]
 [115  12]]

AUC: 0.536673
